In [2]:
import json
import os
from algosdk.v2client import algod
from algosdk import account, mnemonic
from algosdk.transaction import *
#AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn, wait_for_confirmation


In [3]:
# Shown for demonstration purposes. NEVER reveal secret mnemonics in practice.
# Change these values with your mnemonics
mnemonic1 = os.environ.get('mnemonic1')
mnemonic2 = os.environ.get('mnemonic2')
mnemonic3 = os.environ.get('mnemonic3')

# For ease of reference, add account public and private keys to
# an accounts dict.
accounts = {}
counter = 1
for m in [mnemonic1, mnemonic2, mnemonic3]:
    accounts[counter] = {}
    accounts[counter]['sk'] = mnemonic.to_private_key(m)
    accounts[counter]['pk'] = account.address_from_private_key(accounts[counter]['sk'])
    counter += 1

# Specify your node address and token. This must be updated.

algod_address = "http://localhost:4001"
algod_token = "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa"

# Initialize an algod client
algod_client = algod.AlgodClient(algod_token=algod_token, algod_address=algod_address)


In [4]:
#   Utility function used to print created asset for account and assetid
def print_created_asset(algodclient, account, assetid):    
    # note: if you have an indexer instance available it is easier to just use this
    # response = myindexer.accounts(asset_id = assetid)
    # then use 'account_info['created-assets'][0] to get info on the created asset
    account_info = algodclient.account_info(account)
    idx = 0;
    for my_account_info in account_info['created-assets']:
        scrutinized_asset = account_info['created-assets'][idx]
        idx = idx + 1       
        if (scrutinized_asset['index'] == assetid):
            print("Asset ID: {}".format(scrutinized_asset['index']))
            print(json.dumps(my_account_info['params'], indent=4))
            break

In [5]:
#   Utility function used to print asset holding for account and assetid
def print_asset_holding(algodclient, account, assetid):
    # note: if you have an indexer instance available it is easier to just use this
    # response = myindexer.accounts(asset_id = assetid)
    # then loop thru the accounts returned and match the account you are looking for
    account_info = algodclient.account_info(account)
    idx = 0
    for my_account_info in account_info['assets']:
        scrutinized_asset = account_info['assets'][idx]
        idx = idx + 1        
        if (scrutinized_asset['asset-id'] == assetid):
            print("Asset ID: {}".format(scrutinized_asset['asset-id']))
            print(json.dumps(scrutinized_asset, indent=4))
            break

print("Account 1 address: {}".format(accounts[1]['pk']))
print("Account 2 address: {}".format(accounts[2]['pk']))
print("Account 3 address: {}".format(accounts[3]['pk']))

Account 1 address: QSSCWZ7LQYW4ZHW5XVW5ZP77VDEK4YLKWGHIQH2I22HITYBEC3DFYPYU7E
Account 2 address: HVZR6A5UEU24CH3XXCAYEYM4KGQTYCM4LHG6IRYOXR7ODHLCXJMWHKLF7I
Account 3 address: YMN33XNUV5QYNHRN2NNTDLK3F7YY3WJUCHFMOVC5RJNI26X4JJGPCBWGJU


In [6]:
# CREATE ASSET
# Get network params for transactions before every transaction.
params = algod_client.suggested_params()
# comment these two lines if you want to use suggested params
params.fee = 1000
params.flat_fee = True

# Account 1 creates an asset called latinum and
# sets Account 2 as the manager, reserve, freeze, and clawback address.
# Asset Creation transaction

txn = AssetConfigTxn(
    sender=accounts[1]['pk'],
    sp=params,
    total=1000,
    default_frozen=False,
    unit_name="cert",
    asset_name="Certificate",
    manager=accounts[2]['pk'],
    reserve=accounts[2]['pk'],
    freeze=accounts[2]['pk'],
    clawback=accounts[2]['pk'],
    url="https://path/to/my/asset/details", 
    decimals=0)

In [7]:
#sign the transaction
# Sign with secret key of creator
stxn = txn.sign(accounts[1]['sk'])

In [8]:
# Send the transaction to the network and retrieve the txid.
txid = algod_client.send_transaction(stxn)
print(txid)

# Retrieve the asset ID of the newly created asset by first
# ensuring that the creation transaction was confirmed,
# then grabbing the asset id from the transaction.

# Wait for the transaction to be confirmed
wait_for_confirmation(algod_client, txid)

try:
    # Pull account info for the creator
    # account_info = algod_client.account_info(accounts[1]['pk'])
    # get asset_id from tx
    # Get the new asset's information from the creator account
    ptx = algod_client.pending_transaction_info(txid)
    asset_id = ptx["asset-index"]
    print_created_asset(algod_client, accounts[1]['pk'], asset_id)
    print_asset_holding(algod_client, accounts[1]['pk'], asset_id)
except Exception as e:
    print(e)


LW7JOI32MY2Z72GX2K7BN6OXSU7VMXCONF6PXMMBI46GED3GPHJA
Asset ID: 156208991
{
    "clawback": "HVZR6A5UEU24CH3XXCAYEYM4KGQTYCM4LHG6IRYOXR7ODHLCXJMWHKLF7I",
    "creator": "QSSCWZ7LQYW4ZHW5XVW5ZP77VDEK4YLKWGHIQH2I22HITYBEC3DFYPYU7E",
    "decimals": 0,
    "default-frozen": false,
    "freeze": "HVZR6A5UEU24CH3XXCAYEYM4KGQTYCM4LHG6IRYOXR7ODHLCXJMWHKLF7I",
    "manager": "HVZR6A5UEU24CH3XXCAYEYM4KGQTYCM4LHG6IRYOXR7ODHLCXJMWHKLF7I",
    "name": "Certificate",
    "name-b64": "Q2VydGlmaWNhdGU=",
    "reserve": "HVZR6A5UEU24CH3XXCAYEYM4KGQTYCM4LHG6IRYOXR7ODHLCXJMWHKLF7I",
    "total": 1000,
    "unit-name": "cert",
    "unit-name-b64": "Y2VydA==",
    "url": "https://path/to/my/asset/details",
    "url-b64": "aHR0cHM6Ly9wYXRoL3RvL215L2Fzc2V0L2RldGFpbHM="
}
Asset ID: 156208991
{
    "amount": 1000,
    "asset-id": 156208991,
    "is-frozen": false
}
